---
title: "Windpark Kelmarsh (2016) - 12.5 MW"
subtitle: Data Analysis
author: "Jorge A. Thomas"
format: html
jupyter: python3
---


## Data Analysis an Testing Great Tables

Infor available here: 

- https://energymap.co.uk/project.asp?pageid=2948

- https://www.gem.wiki/Kelmarsh_wind_farm


In [ ]:
#| label: init
#| eval: true
#| echo: false

from pathlib import Path
import duckdb
import polars as pl
import polars.selectors as cs
import plotly.express as px
from great_tables import GT, md

PROJECT_ROOT = Path(__file__).parent.parent.parent.parent
folder_path_interim = PROJECT_ROOT / "data" / "interim"

Read parquet files


In [ ]:
tstatus = pl.read_parquet(f"{folder_path_interim}/Kelmarsh_Turbines_Status_20160114_20230109_n385133_cols12.parquet")

duckdb.sql("DESCRIBE tstatus").show()

Filter Stop


In [ ]:
#| label: Filter Stops

stops = pl.sql(    
    """
    SELECT * EXCLUDE ("Custom contract category")    
    FROM tstatus
    WHERE "Status" = 'Stop'
    """).collect()

Filter Forced Outages


In [ ]:
#| label: Filter Forced Outages
# Aggregate tables using SQL syntax

foutages = pl.sql(
    """
    SELECT * 
    FROM stops
    WHERE "IEC category" = 'Forced outage' 
    """).collect()

foutages = foutages.rename({"Duration":"TTR"})

# TTF
foutages = foutages.with_columns(
    (pl.col("Timestamp start").shift(-1) - pl.col("Timestamp end")).shift(1).alias("TTF")
    )

foutages = foutages.with_columns(
    pl.when(pl.col("TTF") < 0).then(None).otherwise(pl.col("TTF")).alias("TTF")
)

foutages.describe

## Time Series for Binary Plot


In [ ]:
#| label: Binary Plot

dtstarts = foutages.select(["Timestamp start", "Status", "Message",  "wt"]).rename({"Timestamp start": "Timestamp"})
dtends = foutages.select(["Timestamp end", "Status", "Message",  "wt"]).with_columns(pl.lit("Start").alias("Status")).rename({"Timestamp end": "Timestamp"})

tstates = pl.concat([dtstarts, dtends]).sort(["wt", 'Timestamp'])

# tstates_ts = dtstarts.join(dtends, on="Timestamp", how="full").sort(["wt", 'Timestamp'])

tstates= tstates.with_columns(
    pl.when(pl.col("Status") == "Stop").then(0).otherwise(1).alias("State"),
    pl.col("Timestamp").dt.year().alias("Year")    
    )

tstates= tstates.with_columns(    
    (pl.col("wt") + " - " + pl.col("Year").cast(str)).alias("wt_Year")
    )

tstates
#TODO: remove missing years!

## Plot Square Signal


In [ ]:
#| label: Square Signal Plot

import plotly.express as px

# Other line_shape options, or interpolation methods between given points:

# 'hv' step ends, equivalent to pyplot's post option;
# 'vh' step starts;
# 'hvh' step middles, x axis;
# 'vhv' step middles, y axis;
# 'spline' smooth curve between points;
# 'linear' line segments between points, default value for line_shape.

fig = px.line(tstates, x='Timestamp', y="State", line_shape='hv', facet_row="wt_Year", color="wt",
     title="Time / State Diagram for Kellmarsh Wind Turbines (2016-2022)",
     subtitle= "(0) Down Step=Forced outage (IEC), (1) Up Step=Start",
     labels={"Timestamp": "Time (UTC)", "State": "", "wt": ""},facet_row_spacing=0.01 )

# Update y-axis to show only 0 and 1 as labels
fig.update_yaxes(
    tickvals=[0, 1],  # Set tick values to 0 and 1
    ticktext=["", ""]  # Optional: Explicitly set tick text
)

fig.update_xaxes(matches=None, tickformat="%b"  )


# Update facet labels orientation and size
fig.for_each_annotation(lambda a: a.update(text=a.text.split("-")[-1],textangle=0, xanchor='left', x=-0.04))  # Make labels horizontal
fig.update_annotations(font_size=8)  # Reduce font size

fig.show()


Add Time Between Failure (TBF) and Time To Failure (TTF)


In [ ]:
#| label: forced outages

foutages_agg_year = (
    foutages.group_by("wt",
                pl.col("Timestamp start").dt.year().alias("Year"))
    .agg(
        pl.len().alias("Total Forced Outages (IEC)"),
        pl.sum("TTF").alias("Total TTF"),        
        pl.sum("TTR").alias("Total TTR"),
        pl.mean("TTR").alias("MTTR"),
        pl.mean("TTF").alias("MTTF")                
        )
    .rename({"wt": "Turbine"})
    .sort("Turbine", "Year") 
    .with_columns(
        (pl.col("MTTR").dt.total_seconds()/3_600 + pl.col("MTTF").dt.total_seconds()/3_600).alias("MTBF"),
        #(pl.col("MTTR").dt.total_hours() + pl.col("MTTF").dt.total_hours()).alias("MTBF hours"),
        #(1/pl.col("MTTF").dt.total_hours()).alias("FOR hours"),
        (1/(pl.col("MTTF").dt.total_seconds()/3_600)).alias("Failure Rate")
        )
    )

foutages_yearly_h = foutages_agg_year.with_columns(    
    pl.col("Total TTR").dt.total_seconds()/3_600,
    pl.col("MTTR").dt.total_seconds()/3_600,
    pl.col("MTTF").dt.total_seconds()/3_600
)

foutages_yearly_h